# 7. Наивный Байес и Классификация текстов

> Байесовские алгоритмы для классификации текстов  
> Извлечение признаков из текста

1. Скачайте этот ноутбук к себе.
2. Заполните пропущенные ячейки, отвечая на заданные вопросы. Там должен быть код! (если не сказано обратное)
3. Сохраните результат в своём гитхаб репозитории.

### Полезная литература

- [Naive Bayes and Text Classification](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html)
- [ODS - классификация](https://habrahabr.ru/company/ods/blog/322534/)
- [Байесовский классификатор](http://www.machinelearning.ru/wiki/index.php?title=%D0%91%D0%B0%D0%B9%D0%B5%D1%81%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80)
- [Интересная статья про спам/не спам](https://habrahabr.ru/post/252265/)

Если вам понравилась задача распознавания текстов, то вот подборочка:

- [библиотека Gensim](https://radimrehurek.com/gensim/)
- [библиотека nltk](http://www.nltk.org/)
- [Лекция про word2vec 🔥](https://www.youtube.com/watch?v=oBb9aFmp0Hs)
- [Лекция Векторные представления слов и документов 🔥](https://www.youtube.com/watch?v=hiDBnEyoZS4)

## Наивный байесовский классификатор

На практике мы использовали его для детекции спам/не спам. В этот раз вы будете классифицировать [новостные тексты](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). Попробуете разные методы  генерации признаков из текстов.

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### 2.1 Загрузка данных (2 балла)

Мы будем использовать только 4 класса текстов: `'alt.atheism', 'sci.space', 'talk.religion.misc', 'comp.graphics'`.

Загрузите тренировочную и тестовую части датасета для этих категорий.

In [60]:
categories = ['alt.atheism','comp.graphics',\
               'sci.space','talk.religion.misc']

In [35]:
data = fetch_20newsgroups(subset='all',
    categories=categories, remove=('headers', 'footers', 'quotes'))#, shuffle=True, random_state=42)

In [36]:
data.target_names

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']

Выведите по 1 тексту для каждой из категории.

In [41]:
import pandas as pd

In [42]:
def to_dataframe(data):

    df = pd.DataFrame([data.data, data.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame(data.target_names)
    targets.columns=['category']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    return out

In [43]:
df = to_dataframe(data)

In [44]:
df.head()

,text,target,category
0,My point is that you set up your views as the ...,0,alt.atheism
9,"\n]The ""corrupted over and over"" theory is pr...",0,alt.atheism
14,"\n\nBut, you wouldn't know what red *was*, and...",0,alt.atheism
26,\n\nWell I agree with you in the sense that th...,0,alt.atheism
27,"\n\nSpecifically, which changes are you talkin...",0,alt.atheism


In [58]:
for cat in df.category.unique():
    row = df[df['category']==cat].head(1)
    print('Category({}): {}\n'.format(row['target'][:1].values[0],cat))  
    print(row['text'][:1].values[0])
    print('-----------------')

Category(0): alt.atheism

My point is that you set up your views as the only way to believe.  Saying 
that all eveil in this world is caused by atheism is ridiculous and 
counterproductive to dialogue in this newsgroups.  I see in your posts a 
spirit of condemnation of the atheists in this newsgroup bacause they don'
t believe exactly as you do.  If you're here to try to convert the atheists 
here, you're failing miserably.  Who wants to be in position of constantly 
defending themselves agaist insulting attacks, like you seem to like to do?!
I'm sorry you're so blind that you didn't get the messgae in the quote, 
everyone else has seemed to.
-----------------
Category(1): comp.graphics


By '8 grey level images' you mean 8 items of 1bit images?
It does work(!), but it doesn't work if you have more than 1bit
in your screen and if the screen intensity is non-linear.

With 2 bit per pixel; there could be 1*c_1 + 4*c_2 timing,
this gives 16 levels, but they are linear if screen intensity

In [52]:
#for i in range(0,4):
#    print('Category: {}\n'.format(four_train.target_names[i]))
#    print("\n".join(four_train.data[i].split("\n")[:3]))
#    print('-----------------')

### 2.2 Мешок слов (6 баллов)

Начнем с самого простого способа извлечения признаков из текстов: обычный подсчет количества употреблений слов в тексте.

Для этого используем [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Настройте его так, чтобы он использовал англиские стоп слова.

Проведите трансформаци текстов из обоих датасетов и выведите размер полученного словаря. (1 векторайзер для тестового и тренировочного датасета! У вас должны получиться матрицы с одинаковыми признаками-словами)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

In [95]:
y = df['target'].astype('int')
X = df['text']

In [63]:
vectorizer = CountVectorizer(stop_words='english')
X_vector = vectorizer.fit_transform(X)

In [55]:
print("Количество слов в словаре: ", len(vectorizer.vocabulary_))
X_vector.shape

Количество слов в словаре:  33529


(3387, 33529)

Будем использовать классифиактор для мультиклассифкации [sklearn.naive_bayes.MultinomialNB]().

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [106]:
from sklearn.model_selection import cross_val_score, train_test_split,GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB

In [116]:
from sklearn.metrics import classification_report, confusion_matrix

In [110]:
params = {
            "alpha": [1,2,3,4,5,6,10], 
         }

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
search = GridSearchCV(MultinomialNB(), params, cv=cv, verbose=2,n_jobs=-1, scoring='accuracy')                    

%time search.fit(X_vector, y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: user 197 ms, sys: 7.1 ms, total: 204 ms
Wall time: 456 ms


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.4s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5, 6, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

In [111]:
print("BEST: score={}, params={}".format(search.best_score_, search.best_params_))

BEST: score=0.8190106447410272, params={'alpha': 1}


In [112]:
X_train, X_test, y_train, y_test = train_test_split(X_vector, y, random_state=42, test_size=0.6, stratify=y)

In [113]:
clf = MultinomialNB(alpha=1)
clf.fit(X_train, np.array(y_train))

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [114]:
y_pred = clf.predict(X_test)

In [120]:
print(confusion_matrix(y_test, y_pred))

[[389   8  27  56]
 [ 19 525  35   5]
 [ 30  35 519   8]
 [154   9  28 186]]


In [118]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.81      0.73       480
           1       0.91      0.90      0.90       584
           2       0.85      0.88      0.86       592
           3       0.73      0.49      0.59       377

    accuracy                           0.80      2033
   macro avg       0.79      0.77      0.77      2033
weighted avg       0.80      0.80      0.79      2033



In [0]:
# Посмотрим на ТОП 10 по важности слова в каждой категории

def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: [%s]" % (category, " ".join(feature_names[top10])))

show_top10(clf, vec, newsgroups_train.target_names)

### 2.3 [Tf-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (6 баллов) 

Мешок слов не учитывает "веса" слов, он просто смотрит их вхождение в документ. Вероятно, было бы полезно взвесить каким-то обраом каждое слово в документе. Действительно, если слово встречается во всех документах, то, наверное, его вес небольшой. А если редкое слово встречается в некоторых документах, то скорее всего оно какое-то узко тематическое.

Один из способов взвесить слова - это использовать меру tf-idf, где:

**TF - term frequency** - частота слова для каждой статьи

$$\LARGE \mathrm{tf}(t,d) = \frac{n_t}{\sum_k n_k}$$

**IDF - inverse document frequency*** — обратная частота документа - уменьшает вес часто встречаемых слов

$$\LARGE \mathrm{idf}(t, D) =  \log \frac{|D|}{|\{\,d_i \in D \mid t \in d_{i}\, \}|}$$

$D$ - число документов в корпусе

$|\{\,d_i \in D \mid t \in d_{i}\, \}|$ - число документов из коллекции ${\displaystyle D}$ , в которых встречается ${\displaystyle t}$  (когда ${\displaystyle n_{t}\neq 0}$ ).

**TF-IDF**

$$\LARGE \operatorname{tf-idf}(t,d,D) = \operatorname{tf}(t,d) \times \operatorname{idf}(t, D)$$


Для него есть класс [sklearn.feature_extraction.text.TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [0]:
show_top10(clf, vec, newsgroups_train.target_names)

### 2.4 В чём проблема? (7 баллов)

Как вы заметили, качество немного улучшилось. Как вы думаете почему у нас до сих пор есть ошибки? В чём проблема? Опирайтесь на вывод топа 10 важных слов, confusion matrix и репорт классификации.

**Задание**: Напишите развернутое рассуждение.

Ответ: ???

![](https://i.imgur.com/3D5pgrk.png)

Можно еще немного улучшить качество предсказания, если использовть n-граммы, то есть не только одиночные слова, но и пары, тройки. Для этого у векторайзера есть параметр `ngram_range`, но будьте осторожны, сгенерируется очень много признаков. Их количество можно ограничить  `max_features`. Ещё можно копать в сторону правильного процессинга текстов: лемматизация и стемминг, чтобы убирать окончания, но это обчно не сильно помогает в английском языке.

Кроме того можно было бы пойти совсем в другую сторону и использовать word-2-vec технологии, тогда бы у нас слова начал "обретать" смысл. Но это вовсем другая история.